# Resample Data to Specific Spatio-Temporal Resolutions 

## xr.datarray.coarsen to resample spatially 
## xr.datarray.resample to resample temporally

In [ ]:
# import libraries
from pathlib import Path
import xarray as xr 
import rioxarray

In [ ]:
#insert the path you have saved the cube or go to tutorial Seasfire_datacube_tutorial.ipynb to see how to access it 

# ds=xr.open_zarr(Path.home() / 'SeasFireCube8daily.zarr')
ds=xr.open_zarr(Path.home() / 'seasfire.zarr')

In [ ]:
ds

In [ ]:
# import csv
# import numpy as np
# from collections import OrderedDict

# with open('min_max_per_var.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile, delimiter=',', quotechar='|')
#     dat_vars = list(ds.data_vars)
#     vars_all = { i*2:str(var_)+"_min" for i, var_ in enumerate(dat_vars)}
#     vars_maxs = { (j*2)+1:str(var_)+"_max" for j, var_ in enumerate(dat_vars)}

#     vars_all.update(vars_maxs)
#     vars_all = OrderedDict(sorted(vars_all.items(), key=(lambda x: int(x[0]))))
#     writer.writerow([vars_all[k] for k in range(len(vars_all))])
#     second_row = []
#     for k, var_ in enumerate(list(ds.data_vars)):
#         print(k,var_)
#         m = float(ds[var_].min(skipna=True))
#         M = float(ds[var_].max(skipna=True))
#         second_row += [m,M]
#         print(m,M)
#         if k==1: break
#     writer.writerow(second_row)



## Spatial aggregation of a variable from 0.25 degrees to 1 degree

In [ ]:
# resampled_t2max=ds['t2mmax'].coarsen(longitude=4,latitude=4).mean()
resampled_coarsen=ds.coarsen(latitude=4,longitude=4).mean()

In [ ]:
# resampled_t2max
resampled_coarsen

In [ ]:
#plot a time frame
#resampled_t2max[1,:,:].plot()

## Resample the coarsen variable temporally to get 3 month averages 

In [ ]:
# monthly_t2max=resampled_t2max.resample(time='3MS').mean()
# resampled_coarsen_year=resampled_coarsen.resample(time='3MS').mean()
year_start = "2007-01-01"
year_end = "2012-01-01"
resampled_coarsen_year=resampled_coarsen.sel(time=slice(year_start, year_end))


In [ ]:
# monthly_t2max
resampled_coarsen_year

In [ ]:
#masked_data=resampled_coarsen_year["BAs_GWIS"]
#masked_data[1,:,:].plot()

In [ ]:
#masked_data[2,:,:].plot()

In [ ]:
size_dict = resampled_coarsen_year.sizes
print(size_dict)

In [ ]:
y1, y2, lat, lon = year_start, year_end, size_dict["latitude"], size_dict["longitude"]
name_= "resampled_cube_8_{}_{}_{}_{}_all.zarr".format(y1,y2,lat,lon) 
print()
resampled_coarsen_year.chunk("auto").to_zarr(Path.home() / name_)

In [ ]:
ds2=xr.open_zarr(Path.home() / name_)
ds2

## Select a specific location

In [ ]:
#select a specific location
#lat, lon doen't have to be the exact same as the lat lon in the dataset 
# it would take the nearest
# ds_sel = monthly_t2max.sel(latitude=19.4721, longitude=-155.5922, method='nearest')

## Select a region 

## Method 1: rioxarray bounding box 

In [ ]:
#select a bounding box 
#first give a crs to the dataset 
#here as well it would take the nearest lat lon 
# l=monthly_t2max.rio.write_crs(4326, inplace=True)

l=resampled_coarsen_year.rio.write_crs(4326, inplace=True)


min_lon = -24.995
min_lat = 28.05
max_lon = 45.50
max_lat = 71.5545

subset = l.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)



In [ ]:
subset 

## Method 2: xarray 

In [ ]:
#vmax is for colorbar maximum value
#time=100
#ds.sel(latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon)).isel(time=time).frpfire.plot(vmax=1)

In [ ]:
#ds_sel_box=ds.sel(latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon)).isel(time=time).frpfire

In [ ]:
#ds_sel_box